In [1]:
from __future__ import (absolute_import, division, print_function, unicode_literals)
import backtrader as bt

In [2]:
# Create a Stratey
class TestStrategy(bt.Strategy):

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close
        
        # To keep track of pending orders
        self.order = None

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, %.2f' % order.executed.price)
            elif order.issell():
                self.log('SELL EXECUTED, %.2f' % order.executed.price)

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # Not yet ... we MIGHT BUY if ...
            if self.dataclose[0] < self.dataclose[-1]:
                    # current close less than previous close

                    if self.dataclose[-1] < self.dataclose[-2]:
                        # previous close less than the previous close

                        # BUY, BUY, BUY!!! (with default parameters)
                        self.log('BUY CREATE, %.2f' % self.dataclose[0])

                        # Keep track of the created order to avoid a 2nd order
                        self.order = self.buy()

        else:

            # Already in the market ... we might sell
            if len(self) >= (self.bar_executed + 5):
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()

In [3]:
# Create a cerebro entity
cerebro = bt.Cerebro()

In [4]:
# Add a strategy
cerebro.addstrategy(TestStrategy)

0

In [5]:
cerebro.broker.setcash(150000.0)

In [6]:
# Create a Data Feed
data = bt.feeds.BacktraderCSVData(dataname='gold-071217-071218.csv')

In [7]:
# Add the Data Feed to Cerebro
cerebro.adddata(data)

In [8]:
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
cerebro.run()
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 150000.00
2018-01-02, Close, 100.72
2018-01-03, Close, 98.78
2018-01-04, Close, 98.08
2018-01-04, BUY CREATE, 98.08
2018-01-05, BUY EXECUTED, 97.70
2018-01-05, Close, 98.29
2018-01-08, Close, 97.71
2018-01-09, Close, 97.05
2018-01-10, Close, 97.90
2018-01-11, Close, 98.05
2018-01-12, Close, 100.45
2018-01-12, SELL CREATE, 100.45
2018-01-16, SELL EXECUTED, 99.08
2018-01-16, Close, 100.39
2018-01-17, Close, 99.30
2018-01-17, BUY CREATE, 99.30
2018-01-18, BUY EXECUTED, 98.27
2018-01-18, Close, 97.85
2018-01-19, Close, 98.65
2018-01-22, Close, 98.90
2018-01-23, Close, 100.03
2018-01-24, Close, 103.35
2018-01-25, Close, 101.20
2018-01-25, SELL CREATE, 101.20
2018-01-26, SELL EXECUTED, 100.89
2018-01-26, Close, 101.25
2018-01-29, Close, 100.71
2018-01-30, Close, 101.09
2018-01-31, Close, 101.16
2018-02-01, Close, 101.00
2018-02-02, Close, 99.23
2018-02-02, BUY CREATE, 99.23
2018-02-05, BUY EXECUTED, 95.29
2018-02-05, Close, 92.63
2018-02-06, Close, 89.16
2018-02-07,